In [30]:
import pandas as pd
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import RSLPStemmer
nltk.download('stopwords')
nltk.download('rslp')
import matplotlib.pyplot as plt
import heapq
from google.colab import drive
import time

# Realizando a leitura do dataset e tokenização

db = pd.read_csv("https://raw.githubusercontent.com/GersonSales/rec-info/master/lab_04/results.csv")

documents = db['text']
stopwords = stopwords.words("portuguese")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [0]:
def buildIndex(documents):
  toker = RegexpTokenizer(r'\b[A-zÀ-ú\d\-\']+')
  inverted_list = {}
  n = 0

  for document in documents:
    n += 1
    tokens = toker.tokenize(document.lower())
    for token in set(tokens):
      if (token not in stopwords and len(token) >= 2):
        count = (n, tokens.count(token))
        if (token not in inverted_list.keys()):
          inverted_list[token] = []
          inverted_list[token].append(count)
        else:
          inverted_list[token].append(count)
  return inverted_list
inverted_index = buildIndex(documents)

In [0]:
inverted_index_df = pd.DataFrame.from_dict(inverted_index, orient="index")
inverted_index_df.to_csv('data.csv')
!cp data.csv result_folder

In [33]:
def documentAtATime(query, inverted_index, k):
  inverted_lists = []
  r = []
  for word in query.split(" "):
    if word in inverted_index.keys():
      inverted_lists.append(inverted_index[word])
  for document in range(1, len(documents)+1):
    sd = 0
    for inverted_list in inverted_lists:
      for i in inverted_list:
        if (i[0] == document):
          sd += i[1]
          break
    heapq.heappush(r, (sd, document))
  return heapq.nlargest(k, r)
    
documentAtATime("presidente", inverted_index, 10)

[(16, 166),
 (15, 63),
 (12, 151),
 (11, 216),
 (11, 19),
 (9, 205),
 (9, 86),
 (8, 25),
 (7, 174),
 (6, 235)]

In [34]:
def termAtATime(query, inverted_index, k):
  a = {}
  inverted_lists = []
  r = []
  for word in query.split(" "):
    if word in inverted_index.keys():
      inverted_lists.append(inverted_index[word])
  for inverted_list in inverted_lists:
    for post in inverted_list:
      d = post[0]
      freq = post[1]
      if (d in a.keys()):
        a[d] += freq
      else:
        a[d] = freq
  for (d, ad) in a.items():
    sd = ad
    heapq.heappush(r, (sd, d))
  return heapq.nlargest(k, r)
    
termAtATime("presidente", inverted_index, 10)

[(16, 166),
 (15, 63),
 (12, 151),
 (11, 216),
 (11, 19),
 (9, 205),
 (9, 86),
 (8, 25),
 (7, 174),
 (6, 235)]

In [0]:
queries = ["lula", "bolsonaro", "preso", "temer", "presidente"]

In [36]:
results_document = []
results_term = []
time_results_document = []
time_results_term = []
k = 10

for query in queries:
  init_document = time.time()
  results_document.append(documentAtATime(query, inverted_index, k))
  end_document = time.time()
  time_results_document.append(end_document - init_document)
  
  init_term = time.time()
  results_term.append(termAtATime(query, inverted_index, k))
  end_term = time.time()
  time_results_term.append(end_term - init_term)

queries_df = pd.DataFrame()
queries_df['query'] = queries
queries_df['document_at_a_time'] = results_document
queries_df['term_at_a_time'] = results_term
queries_df['compare'] = queries_df.document_at_a_time == queries_df.term_at_a_time
queries_df['tempo_médio_document_at_a_time'] = time_results_document
queries_df['tempo_médio_term_at_a_time'] = time_results_term
queries_df

,query,document_at_a_time,term_at_a_time,compare,tempo_médio_document_at_a_time,tempo_médio_term_at_a_time
0,lula,"[(9, 15), (3, 234), (3, 216), (2, 226), (2, 20...","[(9, 15), (3, 234), (3, 216), (2, 226), (2, 20...",True,0.000576,0.000027
1,bolsonaro,"[(46, 151), (37, 166), (35, 207), (26, 19), (1...","[(46, 151), (37, 166), (35, 207), (26, 19), (1...",True,0.001248,0.000077
2,preso,"[(3, 238), (3, 216), (3, 162), (3, 151), (2, 2...","[(3, 238), (3, 216), (3, 162), (3, 151), (2, 2...",True,0.000621,0.000031
3,temer,"[(10, 204), (9, 162), (7, 202), (6, 206), (3, ...","[(10, 204), (9, 162), (7, 202), (6, 206), (3, ...",True,0.000518,0.000026
4,presidente,"[(16, 166), (15, 63), (12, 151), (11, 216), (1...","[(16, 166), (15, 63), (12, 151), (11, 216), (1...",True,0.001889,0.000128
